# Fine Tuning remotely hosted models

<font color=blue>Create a Python Environment locally. Set your API Key as Environment Variable. Use the following commands.</font>

python -m venv YourEnvName<br>
YourEnvName\Scripts\activate.bat<br>
setx AZURE_OPENAI_API_KEY "YOUR_API_KEY_HERE"<br>
<font color=blue>Import Necessary Libraries</font>

In [1]:
import openai
import os

In [3]:
API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 

In [2]:
#openai.api_key = open("key.txt", "r").read().strip("\n")
openai.api_key= API_KEY

#### Upload dataset file

We first upload the dataset file using OpenAI API to generate a File ID which can be used in the fine-tuning API.

In [7]:
#File.create() API is used to upload a file

response = openai.File.create(
  file=open("Tickets_Train.jsonl", "rb"),
  purpose='fine-tune'
)
datafile_id = response.id


In [8]:
print(datafile_id)

file-sBXQlPAhMK5Dv4cz4xB6SKrb


In [3]:
openai.File.list()

<OpenAIObject list at 0x2b3f17933b0> JSON: {
  "data": [
    {
      "bytes": 153498,
      "created_at": 1690130618,
      "filename": "compiled_results.csv",
      "id": "file-y2su5mqCUwKFiEloDwykNny4",
      "object": "file",
      "purpose": "fine-tune-results",
      "status": "processed",
      "status_details": null
    },
    {
      "bytes": 90759,
      "created_at": 1681208226,
      "filename": "file",
      "id": "file-JEq9DsGyplqagSvC7efUEA1C",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    },
    {
      "bytes": 169115,
      "created_at": 1681209151,
      "filename": "compiled_results.csv",
      "id": "file-SjwuD15GWy95AGUzllVGe1ks",
      "object": "file",
      "purpose": "fine-tune-results",
      "status": "processed",
      "status_details": null
    },
    {
      "bytes": 18328,
      "created_at": 1687927489,
      "filename": "file",
      "id": "file-vszL0hzziTMMmKY8Tgt0nNhK",
      "objec

In [10]:
response_v = openai.File.create(
  file=open("Tickets_Validation.jsonl", "rb"),
  purpose='fine-tune'
)
validationfile_id = response_v.id


In [13]:
print(validationfile_id)

file-s1fs0Yggc1dYALLk8feRivTm


In [14]:
openai.File.list()

<OpenAIObject list at 0x2012136d630> JSON: {
  "data": [
    {
      "bytes": 90759,
      "created_at": 1681208226,
      "filename": "file",
      "id": "file-JEq9DsGyplqagSvC7efUEA1C",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    },
    {
      "bytes": 169115,
      "created_at": 1681209151,
      "filename": "compiled_results.csv",
      "id": "file-SjwuD15GWy95AGUzllVGe1ks",
      "object": "file",
      "purpose": "fine-tune-results",
      "status": "processed",
      "status_details": null
    },
    {
      "bytes": 18328,
      "created_at": 1687927489,
      "filename": "file",
      "id": "file-vszL0hzziTMMmKY8Tgt0nNhK",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    },
    {
      "bytes": 72427,
      "created_at": 1690121399,
      "filename": "file",
      "id": "file-sBXQlPAhMK5Dv4cz4xB6SKrb",
      "object": "file",
      "purpos

In [11]:
fine_tuned_model = openai.FineTune.create(training_file=datafile_id,validation_file=validationfile_id) #Here: "file-vszL0hzziTMMmKY8Tgt0nNhK"
#model="curie" by default
#n_epochs=4 by default


We can see that the status of the task is "pending". It is expected to complete over a period of time. We can list the various jobs requested using the following API and check their status. `

In [4]:
openai.FineTune.list()

<OpenAIObject list at 0x2b3f17cf7c0> JSON: {
  "data": [
    {
      "created_at": 1680758322,
      "fine_tuned_model": "curie:ft-infosys-eta-2023-04-06-05-21-55",
      "hyperparams": {
        "batch_size": 1,
        "learning_rate_multiplier": 0.1,
        "n_epochs": 4,
        "prompt_loss_weight": 0.01
      },
      "id": "ft-HGoBOX7yAloq70SB4k6hHmyh",
      "model": "curie",
      "object": "fine-tune",
      "organization_id": "org-lmYuwS6VnhCBVArBveOAnLuA",
      "result_files": [
        {
          "bytes": 777,
          "created_at": 1680758516,
          "filename": "compiled_results.csv",
          "id": "file-nvVyyZYgcXqcBKp4vLRyGy2L",
          "object": "file",
          "purpose": "fine-tune-results",
          "status": "processed",
          "status_details": null
        }
      ],
      "status": "succeeded",
      "training_files": [
        {
          "bytes": 855,
          "created_at": 1680758213,
          "filename": "file",
          "id": "file-DuroS

In [16]:
# Optional BLOCKING wait code to check for status of fine-tuning
'''while fine_tuned_model.status()["data"]["ready"] is False:
    time.sleep(30)
    fine_tuned_model = openai.Model.retrieve(fine_tuned_model.id)

print("Fine-tuning of you model is complete")'''

'while fine_tuned_model.status()["data"]["ready"] is False:\n    time.sleep(30)\n    fine_tuned_model = openai.Model.retrieve(fine_tuned_model.id)\n\nprint("Fine-tuning of you model is complete")'

In [5]:
info = openai.FineTune.list()

In [7]:
print("Fine Tuned Model Name : ", info.data[2].fine_tuned_model)

Fine Tuned Model Name :  curie:ft-infosys-eta-2023-07-23-16-43-37


#### Using the trained model for generating completions

In [ ]:
res_completion=openai.Completion.create(
    model="<your_fine_tuned_model_name>",
    prompt="Need help to install Windows 11.\n\n###\n\n",
    max_tokens=3)
print(res_completion['choices'][0]['text'])


In [8]:
openai.File.retrieve("file-y2su5mqCUwKFiEloDwykNny4")


<File file id=file-y2su5mqCUwKFiEloDwykNny4 at 0x2b3f17cf9f0> JSON: {
  "bytes": 153498,
  "created_at": 1690130618,
  "filename": "compiled_results.csv",
  "id": "file-y2su5mqCUwKFiEloDwykNny4",
  "object": "file",
  "purpose": "fine-tune-results",
  "status": "processed",
  "status_details": null
}

In [9]:
content = openai.File.download("file-y2su5mqCUwKFiEloDwykNny4")


In [14]:
import os
with open("compiled_results.csv", "wb") as binary_file:
    # Write bytes to file
    binary_file.write(content)